# Scraping Hotel Ratings on Tripadvisor

In this homework we will practice web scraping. Let's get some basic information for each hotel in Boston.

On each hotel page, scrape the Traverler ratings. **(10 pts)**

![Information to be scraped](traveler_ratings.png)

Save the data in "traverler_ratings.csv" in the following format:

hotel_name, rating, count

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = "http://www.tripadvisor.com"
city_url = base_url + "/"+ 'Hotels-g60745-Boston_Massachusetts-Hotels.html'
response = requests.get(city_url)
html = response.text.encode('utf-8')
# create a beautifulsoup object of the search page of boston hotel
soup = BeautifulSoup(html, "lxml")

In [4]:
# select the language to be english
li = soup.find('link',{'hreflang':'en'})
url = li['href']
url

'https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html'

In [41]:
# define a function to parse the search page of hotel list 
def parse_hotellist_page(html):

    soup_hotel = BeautifulSoup(requests.get(html).text.encode('utf-8'),'lxml')
    hotel_boxes = soup_hotel.select('div.listing.easyClear.p13n_imperfect')
    
    for hotel_box in hotel_boxes:
        hotel_rating = []
        name = hotel_box.find('div', {'class' :'listing_title'}).find(text=True)
        hotel_url = base_url +  hotel_box.find('div', {'class' :'listing_title'}).find('a')['href']
        # get the url of each hotel in the list of hotels
        hotel_soup = BeautifulSoup(requests.get(hotel_url).text.encode('utf-8'), "lxml")
        review_col = hotel_soup.find('div', {'class':'col rating '})
        review = [tmp.string for tmp in review_col.find_all('div',{'class': 'row_label'})]
        span = review_col.find_all('span', {'class':'row_fill'})
        count = [sp.next_element.next_element.next_element.text.strip('\n') for sp in span]
        for i in range(len(count)):
            hotel_rating.append([name, review[i], count[i]])
        # write ratings and counts to file
        with open('traveler_ratings.csv', 'a', newline='') as csvfile:
            for line in hotel_rating:
                writer = csv.writer(csvfile)
                writer.writerow(line)
    
    # Get next URL page if exists, else exit
    div = soup_hotel.find("div", {"class" : "unified pagination standard_pagination"})
    
    # check if last page
    if div.find('span', {'class' : 'nav next ui_button disabled'}):
        return None
    # If it is not las page there must be the Next URL
    hrefs = div.find_all('a', href= True)
    for href in hrefs:
        if href.find(text = True) == 'Next':
            print ('find next page')
            return href['href']

In [42]:
url

'https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html'

In [43]:
# list1_url = parse_hotellist_page(list_url)
with open('traveler_ratings.csv', 'a') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['hotel_name','rating', 'count'])
j = 0
while(True):    
    j = j + 1
    print (j)
    if j>1:
        url = base_url + url
    url = parse_hotellist_page(url)
    if url==None:
        print ('Write result to travel_rating.csv.')
        break

1
find next page
2
find next page
3
Write result to travel_rating.csv.


-------

Next, scrape all the reviews of each hotel for the star ratings of the following attributes: Value, Location, Sleep Quality, Rooms, Cleanliness, Service. Note that some reviews may not have attribute ratings and some may only have some of the attributes. **(25 pts)**

![Information to be scraped](attribute_ratings.png)

Save the data in "attribute_ratings.csv" in the following format:

hotel_name, review_id, attribute, star_value

In [5]:
li = soup.find('link',{'hreflang':'en'})
url = li['href']
url

'https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html'

In [6]:
def parse_hotellist_reivew(html):
    attr_list = ['Value', 'Location', 'Sleep Quality', 'Rooms', 'Cleanliness', 'Service']
    soup_list = BeautifulSoup(requests.get(html).text.encode('utf-8'),'lxml')
    hotel_boxes = soup_list.select('div.listing.easyClear.p13n_imperfect')
    
    # get each hotel in the list of hotels
    for hotel_box in hotel_boxes:

        name = hotel_box.find('div', {'class' :'listing_title'}).find(text=True)
        # print each hotel name that encountered
        print (name)
        hotel_url = base_url +  hotel_box.find('div', {'class' :'listing_title'}).find('a')['href']
        hotel_soup = BeautifulSoup(requests.get(hotel_url).text.encode('utf-8'), "lxml")
        # get the link of the first review
        if hotel_soup.find('div', {'class': 'quote isNew'})!=None:
            rev_link = base_url + hotel_soup.find('div', {'class': 'quote isNew'}).a['href']
        else:
            rev_link = base_url + hotel_soup.find('div', {'class': 'quote'}).a['href']

        while (True):
            # get the link of the first review to get the star rating, and then traverse the review pages
            rev_soup = BeautifulSoup(requests.get(rev_link).text.encode('utf-8'), "lxml")
            rev_id = [rev.p['id'] for rev in rev_soup.find_all('div', {'class': 'entry'})]

            rev_box = rev_soup.find_all('div',{'class':'rating-list'})
            rating = []
            attr = []
            hotel_rating = []
            for rev in rev_box:
                rate1 = [rate['alt'][0] for rate in rev.find_all('img')]
                rating.append(rate1)
                attribute = [attr.text for attr in rev.find_all('div', {'class': 'recommend-description'})]
                attr.append(attribute)

            for j in range(len(rating)):
                rate = rating[j]
                attribute = attr[j]
                if len(rate) > 0:
                    for jj in range(len(rate)):
                        if attribute[jj] in attr_list:
                            hotel_rating.append([name, rev_id[j], attribute[jj], rate[jj]])
            # write attribute rating to file               
            with open('attribute_ratings.csv', 'a', newline='') as csvfile:
                for line in hotel_rating:
                    writer = csv.writer(csvfile)
                    writer.writerow(line)
            # if there is no more page of reviews, break out of while loop to get the next hotel
            if rev_soup.find('span', {'class': 'nav next disabled'}):
                break
            rev_link = base_url + rev_soup.find("a", {"class" : "nav next rndBtn ui_button primary taLnk"})['href']
    
    # Get next URL page if exists, else exit
    div = soup_list.find("div", {"class" : "unified pagination standard_pagination"})
    
    # check if last page of hotels
    if div.find('span', {'class' : 'nav next ui_button disabled'}):
        return None
    # If it is not las page there must be the Next URL
    hrefs = div.find_all('a', href= True)
    for href in hrefs:
        if href.find(text = True) == 'Next':
            print ('find next page of list of hotels')
            next_page = base_url + href['href']
            return next_page

In [7]:
next_page = url
next_page

'https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html'

In [8]:
with open('attribute_ratings.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['hotel_name','review_id', 'attribute', 'star_value'])
while (True):
    if next_page == None:
        print ('Write result to attribute_ratings.csv.')
        break
    next_page = parse_hotellist_reivew(next_page)
    

Marriott Vacation Club Pulse at Custom House, Boston
Boston Harbor Hotel
Seaport Boston Hotel
Four Seasons Hotel Boston
Lenox Hotel
Courtyard Boston Copley Square
InterContinental Boston
Hotel Commonwealth
Mandarin Oriental, Boston
Residence Inn Boston Back Bay/Fenway
Eliot Hotel
XV Beacon
Residence Inn Boston Downtown/Seaport
Kimpton Nine Zero Hotel
Hilton Garden Inn Boston Logan Airport
The Verb Hotel
The Godfrey Hotel Boston
enVision Hotel Boston - Longwood
The Langham, Boston
Loews Boston Hotel
The Boxer Boston
Colonnade Hotel
Residence Inn Boston Harbor on Tudor Wharf
Fairmont Copley Plaza, Boston
Copley Square Hotel
W Boston
The Bostonian Boston
Taj Boston
The Westin Copley Place, Boston
Newbury Guest House
find next page of list of hotels
The Envoy Hotel, Autograph Collection
Hyatt Regency Boston Harbor
Hyatt Regency Boston
The Ritz-Carlton, Boston
Kimpton Onyx Hotel
Boston Marriott Long Wharf
The Westin Boston Waterfront
Revere Hotel Boston Common
The Inn At St Botolph
Ames Bos

-------